In [19]:
!pip install contractions
!pip install datasets
!pip install Flask flask-ngrok flask-cors

In [20]:
import numpy as np
import pandas as pd

import re
import string # for text cleaning
import contractions # for expanding short form words
from tqdm import tqdm
tqdm.pandas(desc="Progress Bar")

import torch
from datasets import load_dataset
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.metrics.pairwise import cosine_similarity

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:

!ngrok authtoken 2mPZwifpvClOzsoP4f51dmErsBm_6BguCKQ3qY3nztsE7Z1uJ

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [25]:
import firebase_admin
from firebase_admin import credentials, firestore

cred = credentials.Certificate('/content/drive/MyDrive/CV-JD-Matching/serviceAccountKey.json')

firebase_admin.initialize_app(cred)

db = firestore.client()

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

# Get Resume

In [27]:
def get_resume_by_userId(userId):
    resumes_ref = db.collection('Resumes')
    query = resumes_ref.where('userId', '==', userId).get()
    if not query:
        print(f"No resume found for userId: {userId}")
        return None
    for doc in query:
        resume_data = doc.to_dict()
        return resume_data

user_id = "Shruti456"
resume = get_resume_by_userId(user_id)
if resume:
    print(f"Resume for {user_id}: {resume}")
else:
    print("No resume found.")

Resume for Shruti456: {'education': [{'degree': 'Bachelor of Science in Computer Science', 'gradYear': '2022', 'university': 'University of California, Berkeley'}], 'languages': ['English', 'Marathi'], 'parsedText': 'Shruti is a talented frontend engineer with a passion for creating beautiful user interfaces...', 'embeddingVector': [-0.07979843765497208, 0.08942137658596039, 0.07722501456737518, -0.06293588876724243, 0.30689430236816406, -0.2819562256336212, -0.19332920014858246, 0.3673371970653534, -0.11723736673593521, -0.11008210480213165, 0.03504513204097748, -0.3614444136619568, 0.018966952338814735, 0.34477725625038147, 0.055095549672842026, 0.17672695219516754, 0.23545274138450623, 0.12052682042121887, -0.18198886513710022, 0.19299104809761047, 0.010219422169029713, -0.10656534880399704, 0.02201114594936371, 0.491380512714386, 0.22704674303531647, 0.06906893104314804, 0.019056031480431557, 0.23572279512882233, -0.23872889578342438, -0.022725965827703476, 0.32009437680244446, 0.0

In [28]:
import re
import string
import contractions

def text_cleaning(text: str) -> str:
    if not text:
        return ""
    text = text.lower().strip()
    translator = str.maketrans('', '', string.punctuation)
    text = contractions.fix(text)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'\b\d{1,3}[-./]?\d{1,3}[-./]?\d{1,4}\b', '', text)
    text = text.translate(translator)
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    return text.strip()

def process_resume(resume_dict):
    skills = ' '.join(resume_dict.get('skills', []))
    education_list = resume_dict.get('education', [])
    education_text = ' '.join([edu.get('degree', '') for edu in education_list])
    parsed_text = resume_dict.get('parsedText', '')
    combined_text = skills + ' ' + education_text + ' ' + parsed_text
    cleaned_text = text_cleaning(combined_text)
    resume_dict['CV'] = cleaned_text
    return resume_dict


def process_job_description(job_dict):
    job_title = job_dict.get('jobTitle', '')
    job_description = job_dict.get('jobDescription', '')
    responsibilities = ' '.join(job_dict.get('responsibilities', []))
    required_skills = ' '.join(job_dict.get('requiredSkills', []))
    combined_text = f"{job_title} {job_description} {responsibilities} {required_skills}"
    cleaned_text = text_cleaning(combined_text)
    job_dict['CV'] = cleaned_text
    return job_dict

processed_resume = process_resume(resume)
print(f"Cleaned CV for {processed_resume['userId']}: {processed_resume['CV']}")

Cleaned CV for Shruti456: javascript react css html bachelor of science in computer science shruti is a talented frontend engineer with a passion for creating beautiful user interfaces


In [44]:
import torch
from transformers import DistilBertTokenizer, DistilBertModel

def generate_embedding(text: str):
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    model = DistilBertModel.from_pretrained('distilbert-base-uncased')
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    last_hidden_state = outputs.last_hidden_state
    embedding = torch.mean(last_hidden_state, dim=1).squeeze()
    return embedding.tolist()

processed_resume_text = processed_resume['CV']
resume_embedding = generate_embedding(processed_resume_text)
print(f"Resume Embedding for {processed_resume['userId']}: {resume_embedding[:10]}...")

Resume Embedding for Shruti456: [-0.07979843765497208, 0.08942137658596039, 0.07722501456737518, -0.06293588876724243, 0.30689430236816406, -0.2819562256336212, -0.19332920014858246, 0.3673371970653534, -0.11723736673593521, -0.11008210480213165]...


# Flask APIs


In [41]:
# Define the classes as dictionaries for simplicity
class Certification:
    def __init__(self, date, issuer, title):
        self.date = date
        self.issuer = issuer
        self.title = title

class Education:
    def __init__(self, degree, university, gradYear):
        self.degree = degree
        self.university = university
        self.gradYear = gradYear

class Experience:
    def __init__(self, company, role, duration):
        self.company = company
        self.role = role
        self.duration = duration

In [48]:
from flask import Flask, request, jsonify
import firebase_admin
from firebase_admin import credentials, firestore
from pyngrok import ngrok
import uuid

app = Flask(__name__)
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")

def cosine_similarity(vec_a, vec_b):
    dot_product = np.dot(vec_a, vec_b)
    norm_a = np.linalg.norm(vec_a)
    norm_b = np.linalg.norm(vec_b)
    if norm_a == 0 or norm_b == 0:
        return 0.0
    return dot_product / (norm_a * norm_b)


@app.route('/upload_resume', methods=['POST'])
def upload_resume():
    data = request.json
    required_fields = [ 'userId']

    for field in required_fields:
        if field not in data:
            return jsonify({'error': f'Missing field: {field}'}), 400

    try:
        experience_list = [Experience(exp['company'], exp['role'], exp['duration']).__dict__ for exp in data['experience']]
        education_list = [Education(edu['degree'], edu['university'], edu['gradYear']).__dict__ for edu in data['education']]
        certification_list = [Certification(cert['date'], cert['issuer'], cert['title']).__dict__ for cert in data['certifications']]

        resume_data = {
            "resumeId": data['userId'],
            "userId": data['userId'],
            "contactInfo": data['contactInfo'],
            "preferredLocation": data['preferredLocation'],
            "languages": data['languages'],
            "skills": data['skills'],
            "experience": experience_list,
            "education": education_list,
            "certifications": certification_list,
            "fileURL": data['fileURL'],
            "parsedText": data['parsedText'],
            "keywords": data['keywords'],
        }

        processed_resume = process_resume(resume_data)
        resume_embedding = generate_embedding(processed_resume['CV'])
        resume_data["embeddingVector"] = resume_embedding
        db.collection('Resumes').add(resume_data)
        return jsonify({'success': 'Resume uploaded successfully'}), 201
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/upload_job', methods=['POST'])
def upload_job():
    data = request.json
    required_fields = ['employerId']

    for field in required_fields:
        if field not in data:
            return jsonify({'error': f'Missing field: {field}'}), 400
    job_id = str(uuid.uuid4())

    try:
        job_data = {
            "jobId": job_id,
            "employerId": data['employerId'],
            "contactInfo": data['contactInfo'],
            "jobTitle": data['jobTitle'],
            "company": data['company'],
            "jobDescription": data['jobDescription'],
            "requiredSkills": data['requiredSkills'],
            "responsibilities": data['responsibilities'],
            "location": data['location'],
            "salaryRange": data['salaryRange'],
            "keywords": data['keywords'],
            "postDate": data['postDate'],
            "expirationDate": data['expirationDate'],
        }

        processed_job = process_job_description(job_data)
        job_embedding = generate_embedding(processed_job['CV'])
        job_data["embeddingVector"] = job_embedding

        db.collection('Jobs').add(job_data)
        return jsonify({'success': 'Job uploaded successfully'}), 201
    except Exception as e:
        return jsonify({'error': str(e)}), 500


@app.route('/find_top_jobs', methods=['POST'])
def find_top_jobs():
      data = request.json

      if 'userId' not in data:
          return jsonify({'error': 'Missing userId'}), 400
      user_id = data['userId']

      resume_ref = db.collection('Resumes').document(user_id)
      resume_doc = resume_ref.get()

      if resume_doc.exists:
          resume_data = resume_doc.to_dict()
          if 'embeddingVector' not in resume_data or not resume_data['embeddingVector']:
              processed_resume = process_resume(resume_data)
              resume_embedding = generate_embedding(processed_resume['CV'])
              resume_ref.update({'embeddingVector': resume_embedding})
          else:
              resume_embedding = resume_data['embeddingVector']
      else:
          return jsonify({'error': 'Resume not found for userId: {}'.format(user_id)}), 404


      job_collection = db.collection('Jobs').stream()

      similarities = []

      for job in job_collection:
          job_data = job.to_dict()
          job_embedding = job_data.get('embeddingVector')

          if not job_embedding:
            processed_job = process_job_description(job_data)
            job_embedding = generate_embedding(processed_job['CV'])
            db.collection('Jobs').document(str(job_data['jobId'])).update({'embeddingVector': job_embedding})

          score = cosine_similarity(resume_embedding, job_embedding)
          similarities.append((job_data['jobId'], score))

      similarities.sort(key=lambda x: x[1], reverse=True)
      top_jobs = [job_id for job_id, score in similarities[:5]]
      resume_ref = db.collection('Resumes').document(user_id)

      resume_ref.update({
          'matches': top_jobs
      })

      jobs_list = []
      for job_id in top_jobs:
        job_ref = db.collection('Jobs').document(job_id)
        job_doc = job_ref.get()

        if job_doc.exists:
            job_data = job_doc.to_dict()
            job_data.pop('embeddingVector', None)
            jobs_list.append(job_data)
        else:
            return jsonify({'error': 'Job not found for jobId: {}'.format(job_id)}), 404

      return jsonify({'jobs': jobs_list}), 200


@app.route('/find_top_candidates', methods=['POST'])
def find_top_candidates():
    data = request.json

    if 'jobId' not in data:
        return jsonify({'error': 'Missing jobId'}), 400
    job_id = data['jobId']

    # Fetch the job document from Firestore
    job_ref = db.collection('Jobs').document(job_id)
    job_doc = job_ref.get()

    if job_doc.exists:
        job_data = job_doc.to_dict()

        job_embedding = job_data.get('embeddingVector')
        if not job_embedding:
            processed_job = process_job_description(job_data)
            job_embedding = generate_embedding(processed_job['CV'])
            job_ref.update({'embeddingVector': job_embedding})
    else:
        return jsonify({'error': 'Job not found for jobId: {}'.format(job_id)}), 404

    resume_collection = db.collection('Resumes').stream()

    similarities = []

    for resume in resume_collection:
        resume_data = resume.to_dict()
        resume_embedding = resume_data.get('embeddingVector')

        if not resume_embedding:
            processed_resume = process_resume(resume_data)
            resume_embedding = generate_embedding(processed_resume['CV'])
            db.collection('Resumes').document(resume.id).update({'embeddingVector': resume_embedding})

        score = cosine_similarity(job_embedding, resume_embedding)
        similarities.append((resume.id, score))

    similarities.sort(key=lambda x: x[1], reverse=True)
    top_candidates = [candidate_id for candidate_id, score in similarities[:5]]
    candidates_list = []
    for candidate_id in top_candidates:
        # Retrieve each resume from Firestore using candidateId
        resume_ref = db.collection('Resumes').document(candidate_id)
        resume_doc = resume_ref.get()

        if resume_doc.exists:
            candidate_data = resume_doc.to_dict()
            candidate_data.pop('embeddingVector', None)
            candidates_list.append(candidate_data)  # Add the candidate data to the list
        else:
            return jsonify({'error': 'Candidate not found for candidateId: {}'.format(candidate_id)}), 404

    return jsonify({'candidates': candidates_list}), 200


@app.route('/find_top_candidates_by_employer', methods=['POST'])
def find_top_candidates_by_employer():
    data = request.json

    if 'employerId' not in data:
        return jsonify({'error': 'Missing employerId'}), 400
    employer_id = data['employerId']

    job_collection = db.collection('Jobs').where('employerId', '==', employer_id).stream()

    similarities = []

    for job in job_collection:
        job_data = job.to_dict()
        job_embedding = job_data.get('embeddingVector')
        if not job_embedding:
            processed_job = process_job_description(job_data)
            job_embedding = generate_embedding(processed_job['CV'])
            db.collection('Jobs').document(job.id).update({'embeddingVector': job_embedding})

        resume_collection = db.collection('Resumes').stream()

        for resume in resume_collection:
            resume_data = resume.to_dict()
            resume_embedding = resume_data.get('embeddingVector')
            if not resume_embedding:
                processed_resume = process_resume(resume_data)
                resume_embedding = generate_embedding(processed_resume['CV'])
                db.collection('Resumes').document(resume.id).update({'embeddingVector': resume_embedding})
            score = cosine_similarity(job_embedding, resume_embedding)
            similarities.append((resume.id, score))

    similarities.sort(key=lambda x: x[1], reverse=True)

    top_candidates = [candidate_id for candidate_id, score in similarities[:5]]

    candidates_list = []
    for candidate_id in top_candidates:
        resume_ref = db.collection('Resumes').document(candidate_id)
        resume_doc = resume_ref.get()

        if resume_doc.exists:
            candidate_data = resume_doc.to_dict()
            candidate_data.pop('embeddingVector', None)
            candidates_list.append(candidate_data)  # Add the candidate data to the list
        else:
            return jsonify({'error': 'Candidate not found for candidateId: {}'.format(candidate_id)}), 404

    return jsonify({'candidates': candidates_list}), 200


app.run(host='0.0.0.0', port=5000)


Public URL: NgrokTunnel: "https://8512-35-245-143-209.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
/usr/local/lib/python3.10/dist-packages/google/cloud/firestore_v1/base_collection.py:302: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)
INFO:werkzeug:127.0.0.1 - - [22/Sep/2024 15:09:55] "POST /find_top_candidates_by_employer HTTP/1.1" 200 -


# Kaggle JDs

## Loading JD Data

In [30]:
jd_data = load_dataset('jacob-hugging-face/job-descriptions', split="train")
jd_data

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

training_data.csv:   0%|          | 0.00/3.77M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/853 [00:00<?, ? examples/s]

Dataset({
    features: ['company_name', 'job_description', 'position_title', 'description_length', 'model_response'],
    num_rows: 853
})

In [31]:
jd_df = pd.DataFrame(jd_data)
jd_df.head()

,company_name,job_description,position_title,description_length,model_response
0,Google,minimum qualifications\nbachelors degree or eq...,Sales Specialist,2727,"{\n ""Core Responsibilities"": ""Responsible fo..."
1,Apple,description\nas an asc you will be highly infl...,Apple Solutions Consultant,828,"{\n ""Core Responsibilities"": ""as an asc you ..."
2,Netflix,its an amazing time to be joining netflix as w...,Licensing Coordinator - Consumer Products,3205,"{\n ""Core Responsibilities"": ""Help drive bus..."
3,Robert Half,description\n\nweb designers looking to expand...,Web Designer,2489,"{\n ""Core Responsibilities"": ""Designing webs..."
4,TrackFive,at trackfive weve got big goals were on a miss...,Web Developer,3167,"{\n ""Core Responsibilities"": ""Build and layo..."


In [ ]:
# Sample JD

# jd_df['model_response'][0]
print(jd_df['job_description'][0])
print(jd_df['model_response'][0])

minimum qualifications
bachelors degree or equivalent practical experience years of experience in saas or productivity tools businessexperience managing enterprise accounts with sales cycles
preferred qualifications
 years of experience building strategic business partnerships with enterprise customersability to work through and with a reseller ecosystem to scale the businessability to plan pitch and execute a territory business strategyability to build relationships and to deliver results in a crossfunctionalmatrixed environmentability to identify crosspromoting and uppromoting opportunities within the existing account baseexcellent account management writtenverbal communication strategic and analyticalthinking skills
about the job
as a member of the google cloud team you inspire leading companies schools and government agencies to work smarter with google tools like google workspace search and chrome you advocate the innovative power of our products to make organizations more product

## Loading Extracted CV Data

In [34]:
df = pd.read_csv('/content/drive/MyDrive/CV-JD-Matching/pdf_extracted_skills_education.csv')
df.head()

,Skills,Education,ID,Category
0,Accounting; General Accounting; Accounts Payab...,Northern Maine Community College 1994 Associat...,10554236,ACCOUNTANT
1,"accounting, accounts payable, Accounts Receiva...","Bachelor of Science : Accounting , May 2010 Un...",10674770,ACCOUNTANT
2,"accounts payables, accounts receivables, Accou...",Computer Applications Specialist Certificate P...,11163645,ACCOUNTANT
3,"accounting, balance sheet, budgets, client, cl...","EMORY UNIVERSITY, Goizueta Business School 5 2...",11759079,ACCOUNTANT
4,Aderant/CMS Financial reporting,Bachelor of Business Administration : Accounti...,12065211,ACCOUNTANT


In [35]:
df.shape

(2484, 4)

## Basic text cleaning

And doing some preprocessing in the data as discussed in the EDA notebook

In [36]:
def text_cleaning(text:str) -> str:
    if pd.isnull(text):
        return

    # lower-case everything
    text = text.lower().strip()

    # For removing puctuations
    translator = str.maketrans('', '', string.punctuation)

    # expand all the short-form words
    text = contractions.fix(text)

    # remove any special chars
    text = re.sub(r'http\S+|www\S+|https\S+', '', text) # Remove URLs
    text = re.sub(r'\S+@\S+', '', text) # Remove emails
    text = re.sub(r'\b\d{1,3}[-./]?\d{1,3}[-./]?\d{1,4}\b', '', text) # Remove phone numbers
    text = text.translate(translator) # Remove puctuations
    text = re.sub(r'[^a-zA-Z]', ' ', text) # Remove other non-alphanumeric characters

    return text.strip()

In [37]:
# We have 15 Resumes where Skills & Education were not extracted -- REFER EDA notebook
# So, let's remove them
cv_df = df[~(df['Skills'].isna() & df['Education'].isna())].reset_index(drop=True)

# Filling the null values in Skills & Education with Empty String before concatinating them
cv_df = cv_df.fillna(value='')

# Let's stitch together Skills & Education, similar to given in job description.
cv_df['CV'] = cv_df['Skills'] + ' ' + cv_df['Education']

# Doing text cleaning
cv_df['CV'] = cv_df['CV'].progress_apply(text_cleaning)

Progress Bar: 100%|██████████| 2469/2469 [00:01<00:00, 2190.43it/s]


In [ ]:
cv_df.shape

(2469, 5)

In [38]:
# Sample job descriptions
job_descriptions = jd_df['job_description'].apply(text_cleaning)[:15].to_list() # jd_df['job_description'][:15]

# Sample resumes (replace with your extracted resume data)
resumes = cv_df['CV'].to_list()

In [ ]:
print(resumes[0])



accounting general accounting accounts payable program management northern maine community college  associate  accounting city  state  usa emphasis in business  associates  accounting city  state  usa gpa gpa  accounting gpa   hours quarter attended husson college major accounting    semester hours toward bachelors degree professional military comptroller school  wk  managerial accounting i  interestedbased bargaining training for management   hrs  auditing methods and concepts  organizational leadership   hrs  management development ii   hrs


In [ ]:
print(job_descriptions[0])

minimum qualifications bachelors degree or equivalent practical experience years of experience in saas or productivity tools businessexperience managing enterprise accounts with sales cycles preferred qualifications  years of experience building strategic business partnerships with enterprise customersability to work through and with a reseller ecosystem to scale the businessability to plan pitch and execute a territory business strategyability to build relationships and to deliver results in a crossfunctionalmatrixed environmentability to identify crosspromoting and uppromoting opportunities within the existing account baseexcellent account management writtenverbal communication strategic and analyticalthinking skills about the job as a member of the google cloud team you inspire leading companies schools and government agencies to work smarter with google tools like google workspace search and chrome you advocate the innovative power of our products to make organizations more product

## Creating Embedding using `DistilBertTokenizer`, `DistilBertModel`

In [39]:
%%time

# Initialize the DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')


# Tokenize and embed job descriptions
job_description_embeddings = []
for description in job_descriptions:
    tokens = tokenizer(description, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        output = model(**tokens)
    embeddings = output.last_hidden_state.mean(dim=1).numpy()
    job_description_embeddings.append(embeddings[0])  # Flatten the embeddings to 1D


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


CPU times: user 15.1 s, sys: 2.2 s, total: 17.3 s
Wall time: 21.8 s


In [ ]:
# Tokenize and embed resumes
resume_embeddings = []
for resume in resumes:
    tokens = tokenizer(resume, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        output = model(**tokens)
    embeddings = output.last_hidden_state.mean(dim=1).numpy()
    resume_embeddings.append(embeddings[0])  # Flatten the embeddings to 1D

In [ ]:
save_directory = '/content/drive/MyDrive/distilbert_model'

# Save the model and tokenizer
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

('/content/drive/MyDrive/distilbert_model/tokenizer_config.json',
 '/content/drive/MyDrive/distilbert_model/special_tokens_map.json',
 '/content/drive/MyDrive/distilbert_model/vocab.txt',
 '/content/drive/MyDrive/distilbert_model/added_tokens.json')

In [ ]:
job_description_embeddings[0].shape, resume_embeddings[0].shape

((768,), (768,))

In [ ]:
len(job_description_embeddings), len(resume_embeddings)

(15, 2469)

## Calculating Similarity Score & Getting Top 5 Candidates

In [ ]:
# Calculate cosine similarity between job descriptions and resumes
similarity_scores = cosine_similarity(job_description_embeddings, resume_embeddings)
similarity_scores

array([[0.82088774, 0.77677274, 0.7848917 , ..., 0.8589794 , 0.7705459 ,
        0.60554785],
       [0.7844252 , 0.7171359 , 0.7481307 , ..., 0.8324866 , 0.7884388 ,
        0.6784491 ],
       [0.81398576, 0.7895926 , 0.78363687, ..., 0.84983325, 0.7580299 ,
        0.6202629 ],
       ...,
       [0.81677645, 0.7708843 , 0.7748317 , ..., 0.884326  , 0.7708575 ,
        0.6345933 ],
       [0.8439074 , 0.77827585, 0.7971461 , ..., 0.87831897, 0.82519805,
        0.68138146],
       [0.83300304, 0.7873483 , 0.77523583, ..., 0.88907975, 0.7842255 ,
        0.64485663]], dtype=float32)

In [ ]:
# Rank job descriptions for each candidate based on similarity scores
num_top_jobs = 5
top_jobs_for_candidates = []

for candidate_index in range(len(resumes)):  # Loop through each candidate
    # Get the similarity scores for the current candidate across all job descriptions
    job_scores = list(enumerate(similarity_scores[:, candidate_index]))  # Get scores for the candidate
    # Sort job descriptions based on similarity scores in descending order
    job_scores.sort(key=lambda x: x[1], reverse=True)
    # Select the top job descriptions
    top_jobs_for_candidate = job_scores[:num_top_jobs]
    top_jobs_for_candidates.append(top_jobs_for_candidate)

# Print the top job descriptions for each candidate
for candidate_index in range(len(resumes)):
    print(f"Top job descriptions for Candidate {candidate_index + 1} - {cv_df['Category'][candidate_index]}/{cv_df['ID'][candidate_index]}.pdf")
    for job_index, score in top_jobs_for_candidates[candidate_index]:
        print(f"  JD {job_index + 1} - Similarity Score: {score:.4f} - Position: {jd_df['position_title'][job_index]}")
    print()


Streaming output truncated to the last 5000 lines.
  JD 11 - Similarity Score: 0.8181 - Position: Web Developer

Top job descriptions for Candidate 1756 - FITNESS/29165698.pdf
  JD 8 - Similarity Score: 0.8596 - Position: Web Designer
  JD 4 - Similarity Score: 0.8543 - Position: Web Designer
  JD 10 - Similarity Score: 0.8503 - Position: SR. Web Designer
  JD 6 - Similarity Score: 0.8449 - Position: Frontend Web Developer
  JD 14 - Similarity Score: 0.8443 - Position: Wordpress Web Developer

Top job descriptions for Candidate 1757 - FITNESS/29306433.pdf
  JD 14 - Similarity Score: 0.7665 - Position: Wordpress Web Developer
  JD 4 - Similarity Score: 0.7414 - Position: Web Designer
  JD 8 - Similarity Score: 0.7406 - Position: Web Designer
  JD 10 - Similarity Score: 0.7315 - Position: SR. Web Designer
  JD 12 - Similarity Score: 0.7309 - Position: Web Developer

Top job descriptions for Candidate 1758 - FITNESS/29319314.pdf
  JD 14 - Similarity Score: 0.8529 - Position: Wordpress Web

In [ ]:
# Rank candidates for each job description based on similarity scores
num_top_candidates = 5
top_candidates = []

for i, job_description in enumerate(job_descriptions):
    candidates_with_scores = list(enumerate(similarity_scores[i]))
    candidates_with_scores.sort(key=lambda x: x[1], reverse=True)
    top_candidates_for_job = candidates_with_scores[:num_top_candidates]
    top_candidates.append(top_candidates_for_job)

# Print the top candidates for each job description
for i, job_description in enumerate(job_descriptions):
    print(f"Top candidates for JD {i+1} - Postition: {jd_df['position_title'][i]}")
    for candidate_index, score in top_candidates[i]:
        print(f"  Candidate {candidate_index + 1} - Similarity Score: {score:.4f} - {cv_df['Category'][candidate_index]}/{cv_df['ID'][candidate_index]}.pdf")
        # print(f"  Resume: {resumes[candidate_index]}")
    print()

Top candidates for JD 1 - Postition: Sales Specialist
  Candidate 1949 - Similarity Score: 0.9415 - HR/18827609.pdf
  Candidate 291 - Similarity Score: 0.9388 - AGRICULTURE/62994611.pdf
  Candidate 478 - Similarity Score: 0.9387 - ARTS/43622023.pdf
  Candidate 28 - Similarity Score: 0.9377 - ACCOUNTANT/16237710.pdf
  Candidate 1803 - Similarity Score: 0.9314 - HEALTHCARE/10466208.pdf

Top candidates for JD 2 - Postition: Apple Solutions Consultant
  Candidate 168 - Similarity Score: 0.9236 - ADVOCATE/22391901.pdf
  Candidate 904 - Similarity Score: 0.9165 - BUSINESS-DEVELOPMENT/95382114.pdf
  Candidate 1730 - Similarity Score: 0.9159 - FITNESS/21238396.pdf
  Candidate 952 - Similarity Score: 0.9155 - CHEF/21869994.pdf
  Candidate 482 - Similarity Score: 0.9146 - ARTS/54100393.pdf

Top candidates for JD 3 - Postition: Licensing Coordinator - Consumer Products
  Candidate 478 - Similarity Score: 0.9598 - ARTS/43622023.pdf
  Candidate 2153 - Similarity Score: 0.9496 - PUBLIC-RELATIONS/122

In [ ]:
!pip install flask pyngrok


In [ ]:
!ngrok authtoken 2mPZVfVK2W0PSQ9TBsfaKkF0M49_7sCo4vBUvMQYB6em2D4DF


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from flask import Flask
from pyngrok import ngrok

# Initialize the Flask app
app = Flask(__name__)

# Define a sample route
@app.route('/')
def hello_world():
    return "Hello, World!"

# Start the Flask app using ngrok
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")

# Run the app
app.run(host='0.0.0.0', port=5000)


Public URL: NgrokTunnel: "https://d10f-104-155-132-106.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
